In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import plotly.express as px
import base64

import numpy as np
import pandas as pd

from dash import dcc, html, dash_table, Input, Output
from dash.dependencies import Input, Output
from pymongo import MongoClient
from AnimalShelterScript import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "T.13.revor_"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

# Image
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#########################
# Dashboard Layout / View
#########################
app = dash.Dash(__name__)

app.layout = html.Div([
    html.A([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width="250", height="150"),    
    ], href="https://www.snhu.edu"),
    html.Center(html.B(html.H1('Trevor Mickelson SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    # Handling page size selection
    html.Div(className='row', style={'flex-wrap': 'nowrap'},
         children=[dcc.Dropdown(id='dropdown-button-pages', options=[
             {"label": "Page Size 10", "value": "Reset"},
             {"label": "One", "value": 1},
             {"label": "Two", "value": 2},
             {"label": "Three", "value": 3},
             {"label": "Four", "value": 4},
             {"label": "Five", "value": 5},
             {"label": "Six", "value": 6},
             {"label": "Seven", "value": 7},
             {"label": "Eight", "value": 8},
             {"label": "Nine", "value": 9},
         ], value='Reset', clearable=False )]),
    
    # Handling type selection
    html.Div(className='row', style={'flex-wrap': 'nowrap'},
     children=[dcc.Dropdown(id='dropdown-button-types', options=[
         {"label": "All Rescue Types", "value": "Reset"},
         {"label": "Water Rescue", "value": "Water Rescue"},
         {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
         {"label": "Disaster Rescue", "value": "Disaster Rescue"},
     ], value='Reset', clearable=False )]),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                    ),
                 html.Div(
                    id='map-id',
                    className='col s12 m6',
                    )
             ])
])

@app.callback(
    Output('datatable-id', 'data'),
    [Input('dropdown-button-types', 'value')])
def on_click(rescue_type):
    if rescue_type == "Water Rescue": 
        df = pd.DataFrame.from_records(shelter.read({'$and':
            [{'$or': [
            {"breed": {'$regex': "Labrador Retriever"}}, 
            {"breed": {'$regex': "Chesa Bay Retr"}}, 
            {"breed": {'$regex': "Newfoundland"}}
            ]},
            {"sex_upon_outcome": "Intact Female"},
            {"age_upon_outcome_in_weeks": {'$gt': 26, '$lt': 156}}

            ]}))            
    elif rescue_type == "Mountain or Wilderness Rescue":
        df = pd.DataFrame.from_records(shelter.read({'$and':
            [{'$or': [
            {"breed": {'$regex': "German Shepherd"}}, 
            {"breed": {'$regex': "Alaskan Malamute"}}, 
            {"breed": {'$regex': "Old English Sheepdog"}},
            {"breed": {'$regex': "Siberian Husky"}},
            {"breed": {'$regex': "Rottweiler"}}
            ]},
            {"sex_upon_outcome": "Intact Female"},
            {"age_upon_outcome_in_weeks": {'$gt': 26, '$lt': 156}}  
            ]}))
        
    elif rescue_type == "Disaster Rescue":
        df = pd.DataFrame.from_records(shelter.read({'$and':
            [{'$or': [
            {"breed": {'$regex': "Dobberman Pinscher"}}, 
            {"breed": {'$regex': "German Shepherd"}}, 
            {"breed": {'$regex': "Golden Retriever"}},
            {"breed": {'$regex': "Bloodhound"}},
            {"breed": {'$regex': "Rottweiler"}}
            ]},
            {"sex_upon_outcome": "Intact Female"},
            {"age_upon_outcome_in_weeks": {'$gt': 20, '$lt': 300}}
            ]}))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    return df.to_dict('records')

@app.callback(
    Output('datatable-id', 'page_size'),
    [Input('dropdown-button-pages', 'value')])
def on_click(page_number):
    if page_number == "Reset":
        return 10
    else:
        return page_number
    
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
       dcc.Graph(            
           figure = px.pie(
               dff,
               values='age_upon_outcome_in_weeks',
               names='breed'
           )
       )    
    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data")])
def update_map(viewData):    
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app.run_server()